In [139]:
import pandas as pd
import numpy as np
import sqlite3
import importlib

# keeping company information in additional file
import data_file

In [ ]:
importlib.reload(data_file)

In [140]:
targets = pd.read_excel('data_files/targets.xlsx')
targets['tm_check'] = 'no'
targets['pure tm'] = 'no'

In [141]:
targets_fy21 = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY21', 'tm_check', 'pure tm']]
targets_fy21 = targets_fy21.dropna(subset=['FY21'])
targets_fy21 = targets_fy21[targets_fy21['FY21'] != 0]
targets_fy21['FY21'] = targets_fy21['FY21'] / 12
targets_fy21['FY'] = 2021
targets_fy21 = targets_fy21.rename(columns={'FY21': 'target_order_intake_amount_eur'})
num_fy21 = len(targets_fy21)

targets_fy22 = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY22', 'tm_check', 'pure tm']]
targets_fy22 = targets_fy22.dropna(subset=['FY22'])
targets_fy22 = targets_fy22[targets_fy22['FY22'] != 0]
targets_fy22['FY22'] = targets_fy22['FY22'] / 12
targets_fy22['FY'] = 2022
targets_fy22 = targets_fy22.rename(columns={'FY22': 'target_order_intake_amount_eur'})
num_fy22 = len(targets_fy22)

targets_fy23_pci = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY23_PCI', 'tm_check', 'pure tm']]
targets_fy23_pci = targets_fy23_pci.dropna(subset=['FY23_PCI'])
targets_fy23_pci = targets_fy23_pci[targets_fy23_pci['FY23_PCI'] != 0]
targets_fy23_pci['FY23_PCI'] = targets_fy23_pci['FY23_PCI'] / 12
targets_fy23_pci['FY'] = 2023
targets_fy23_pci = targets_fy23_pci.rename(columns={'FY23_PCI': 'target_order_intake_amount_eur'})
num_fy23_p = len(targets_fy23_pci)

targets_fy23_sys = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY23_Systems', 'tm_check', 'pure tm']]
targets_fy23_sys = targets_fy23_sys.dropna(subset=['FY23_Systems'])
targets_fy23_sys = targets_fy23_sys[targets_fy23_sys['FY23_Systems'] != 0]
targets_fy23_sys['FY23_Systems'] = targets_fy23_sys['FY23_Systems'] / 12
targets_fy23_sys['FY'] = 2023
targets_fy23_sys = targets_fy23_sys.rename(columns={'FY23_Systems': 'target_order_intake_amount_eur'})
num_fy23_s = len(targets_fy23_sys)

In [142]:
fy_21 =  [202104, 202105, 202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203]
fy_22 =  [202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211, 202212, 202301, 202302, 202303]
fy_23 =  [202304, 202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403]

In [143]:
targets_fy21 = pd.concat([targets_fy21] * 12, ignore_index=True)
targets_fy21 = targets_fy21.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy21['year_month'] = fy_21 * num_fy21
targets_fy21['for_bu'] = 'PCI Others'
targets_fy21['tm_check'] = 'no'
targets_fy21['tm_check'] = 'no'

targets_fy22 = pd.concat([targets_fy22] * 12, ignore_index=True)
targets_fy22 = targets_fy22.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy22['year_month'] = fy_22 * num_fy22
targets_fy22['for_bu'] = 'PCI Others'

targets_fy23_pci = pd.concat([targets_fy23_pci] * 12, ignore_index=True)
targets_fy23_pci = targets_fy23_pci.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy23_pci['year_month'] = fy_23 * num_fy23_p
targets_fy23_pci['for_bu'] = 'PCI Others'

targets_fy23_sys = pd.concat([targets_fy23_sys] * 12, ignore_index=True)
targets_fy23_sys = targets_fy23_sys.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy23_sys['year_month'] = fy_23 * num_fy23_s
targets_fy23_sys['for_bu'] = 'Systems&ADS'

In [144]:
combined_df = pd.concat([targets_fy21, targets_fy22, targets_fy23_pci, targets_fy23_sys], ignore_index=True)

combined_df['FY'] = (pd.to_datetime(combined_df['year_month'], format='%Y%m') - pd.DateOffset(months=3)).dt.strftime('%d-%m-%Y')

In [146]:
writer = pd.ExcelWriter('data_files/outcome/target.xlsx')
# Save each DataFrame to a separate sheet in the same file
combined_df.to_excel(writer, sheet_name='target', index=False)
# Save the file
writer.close()